<a href="https://colab.research.google.com/github/ichekhovskikh/word-movers-distance/blob/master/word_movers_distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pymorphy2[fast]

import gensim
import os
import collections
import smart_open
import random
import json
import urllib.request
import pymorphy2
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from pymystem3 import Mystem
from gensim.models import Word2Vec
from gensim.similarities import WmdSimilarity

#Начинаем
Для начала нам понадобится комплект документов для обучения нашей модели doc2vec. Теоретически, документ может быть чем угодно: коротким твитом из 140 символов, отдельным абзацем, новостной статьей или книгой. В NLP комплект документов часто называют корпусом.

Будем тренировать нашу модель на собственном корпусе. Этот корпус содержит 500 научных статей на 5 различных тем.

Для тестирования будет использоваться тестовый корпус из 50 статей (10 статей на каждую тему).

Dataset состоит из трех строк: id (идентификатор строки), text (текст статьи), tag (идентификатор самой статьи, вектор которого будем обучать), class_name (был размечен вручную, необходим только для тестирования)

In [0]:
#@title Введите путь к файлам исходной базы статей:
train_path = 'https://raw.githubusercontent.com/ichekhovskikh/cyberleninka-article-downloader/master/train_corpus.json' #@param {type: "string"}
test_path = 'https://raw.githubusercontent.com/ichekhovskikh/cyberleninka-article-downloader/master/test_corpus.json' #@param {type: "string"}

## Опредлим функцию для чтения и предварительной обработки текста
Ниже мы определяем функцию для открытия  train/test файла, предварительно обрабатываем каждый текст датасета, используя простой инструмент предварительной обработки gensim (то есть, разбиваем текст на отдельные слова, удалите знаки препинания, установите строчные буквы и т. д.), лемматизацию, удаление стоп слов и возвращаем список слов. Для обучения модели нам нужно будет связать тег с каждым документом учебного корпуса. В нашем случае тег - это идентификатор статьи.

Лемматизация каждого слова статьи:

In [0]:
morph = pymorphy2.MorphAnalyzer()

def lemmatize(words):
    for word in words:
        yield morph.parse(word)[0].normal_form

Удаление стоп слов:

In [0]:
russian_stopwords = stopwords.words("russian")

def remove_stopwords(words):
    return [word for word in words if word not in russian_stopwords]

Предобработка текста статьи:

In [0]:
def advanced_preprocess(text):
    normalized_text = gensim.utils.simple_preprocess(text)
    normalized_text = list(lemmatize(normalized_text))
    normalized_text = remove_stopwords(normalized_text)
    return normalized_text

Отрытие файла с корпусом статей:

In [0]:
def read_corpus(corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        return [advanced_preprocess(article['text']) for article in corpus]

Получение исходного текста статьи по индексу:

In [0]:
def get_article_text_by_index(index, corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        return corpus[index]['text']

Получение категории статьи по индексу:

In [0]:
def get_article_class_by_index(index, corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        return corpus[index]['class_name']

Загружаем корпуса:

In [0]:
train_corpus = read_corpus(train_path)
test_corpus = read_corpus(test_path)

Давайте посмотрим на учебный корпус:

In [0]:
print(train_corpus[:2])

Корпус тестирования выглядит так:

In [0]:
print(test_corpus[:2])

# Обучение модели
## Создание объекта WmdSimilarity
Теперь мы создадим модель Woc2Vec с векторным размером 120 слов и перебираем учебный корпус 500 раз (данные параметры были получены после проведения ряда исследований).

Словарь содержит в себе все уникальные слова, извлеченных из учебного корпуса.
Параметры обучения:
- sentences: список текстов
- size: размерность вектора
- window: максимальное расстояние между текущим и прогнозируемым словом в предложении
- sg: алгоритм обучения (1 - skip-gram, 0 - CBOW)
- alpha: коэфициент обучения
- iter: количество эпох.

In [0]:
#@title Укажите параметры обучения модели:
vector_size = 120 #@param
window =  6 #@param
epochs =  500 #@param
alpha = 0.001 #@param
learning_method = "skip-gram" #@param ["skip-gram", "CBOW"] {type:"string"}

sg = 1
if (learning_method == "CBOW"):
    sg = 0

In [0]:
%time model = Word2Vec(train_corpus, size=vector_size, window=window, alpha=alpha, iter=epochs, sg=sg, workers=4)

Теперь будем использовать обученный Word2Vec для нахождения близости тектов на основе Word Mover's Distance.

In [0]:
wmd_similarity = WmdSimilarity(train_corpus, model, num_best=5)

# Оценочная модель
Чтобы оценить нашу новую модель, мы сначала выведем новые векторы для каждого документа тренировочного корпуса, сравним выведенные векторы с тренировочным корпусом.

Проверка выведенного вектора по обучающему вектору является своего рода «проверкой работоспособности» в отношении того, ведет ли модель себя адекватно, хотя и не является реальным значением «точности».

Можем взглянуть на пример:

In [0]:
doc_id = random.randint(0, len(train_corpus) - 1)
doc = train_corpus[doc_id]
sims = wmd_similarity[doc]
print('ТЕКСТ ИСХДНОГО ДОКУМЕНТА ({}): «{}»\n'.format(doc_id, get_article_text_by_index(doc_id, train_path)))
num = 0
for index, similarity in sims:
    num += 1
    print(u'%s) %s: «%s»\n' % (num, similarity, get_article_text_by_index(index, train_path)))

# Тестирование модели
Используя тот же подход, что и выше, мы выведем вектор для случайно выбранного тестового документа и сравним документ с нашей моделью на глаз.

In [0]:
from sklearn.metrics import f1_score

test_size = len(test_corpus)

predicted_classes = [] 
test_classes = []

for doc_index in range(test_size):
    doc = test_corpus[doc_index]
    sims = wmd_similarity[doc]
    
    test_article_class = get_article_class_by_index(doc_index, test_path)
    print('ТЕКСТ ИСХДНОГО ДОКУМЕНТА ({}) «{}»: «{}»\n'.format(doc_index, test_article_class, get_article_text_by_index(doc_index, test_path)))
    num = 0
    for index, similarity in sims:
        num += 1
        predicted_article_class = get_article_class_by_index(index, train_path)
        print(u'%s) %s «%s»: «%s»\n' % (num, similarity, predicted_article_class, get_article_text_by_index(index, train_path)))
        predicted_classes.append(predicted_article_class)
        test_classes.append(test_article_class)
        
print('f1score: {}'.format(f1_score(test_classes, predicted_classes, average='macro')))

Тестирование на малых данных для выявления ошибок в ходе написания кода

In [0]:
from gensim.test.utils import common_texts

text = advanced_preprocess("test interface minors")

print('common texts = ', common_texts)
print('text = ', text)

model = Word2Vec(common_texts, size=300, window=5, min_count=1, iter=10, workers=4)
wmd_similarity = WmdSimilarity(common_texts, model, num_best=10)


sims = wmd_similarity[text]
num = 0
for index, similarity in sims:
    num += 1
    print(u'%s) %s: «%s»\n' % (num, similarity, common_texts[index]))


# Поиск похожих научных документов
Выполните поиск похожих научных статей

In [0]:
#@title Укажите путь к тексту статьи в формате *.txt или введите текст статьи:
article_text = '' #@param {type: "string"}
article_path = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/PY0101EN/labs/example1.txt' #@param {type: "string"}

if (article_text == ''):
    with urllib.request.urlopen(article_path) as article_url:
      article_text = article_url.read().decode()


In [0]:
normalized_text = advanced_preprocess(article_text)
sims = wmd_similarity[normalized_text]

print('ТЕКСТ ИСХДНОГО ДОКУМЕНТА «{}»\n'.format(article_text)
num = 0
for index, similarity in sims:
    num += 1
    print(u'%s) %s: «%s»\n' % (num, similarity, get_article_text_by_index(index, train_path)))